In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d as conv2

from skimage import color, data, restoration, io, img_as_float
from skimage.restoration import uft
from scipy.signal import fftconvolve, convolve, convolve2d
from scipy.stats.stats import pearsonr
from numpy.fft import fftn, ifftn
from skimage.measure import compare_psnr
from skimage.draw import bezier_curve

In [ ]:
astro = color.rgb2gray(data.astronaut())

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(astro, 'gray')
plt.show()

In [ ]:
def edgetaper(I, psf):
    """
    J = EDGETAPER(I,PSF) blurs the edges of image I using the point-spread
    function, PSF. The output image J is the weighted sum of the original
    image I and its blurred version. The weighting array, determined by the
    autocorrelation function of PSF, makes J equal to I in its central
    region, and equal to the blurred version of I near the edges.
    
    The EDGETAPER function reduces the ringing effect in image deblurring
    methods that use the discrete Fourier transform, such as DECONWNR,
    DECONVREG, and DECONVLUCY.
    
    Note that the size of the PSF cannot exceed half of the image size in any
    dimension.
    """
    # [I, PSF, sizeI, classI, sizePSF, numNSdim] = parse_inputs(varargin{:});
    if np.all(psf >= 0):
        psf /= psf.sum()
    # PSF size cannot be larger than sizeI/2 because windowing is performed
    # with PSF autocorrelation function
    if np.any(np.array(I.shape) <= np.array(psf.shape) * 2):
        raise ValueError(psf)

    numDim = len(psf.shape)
    beta = [0] * numDim
    for n in range(numDim):
        PSFproj = np.sum(psf, axis=n)
        z = np.real(np.fft.ifftn(np.abs(np.fft.fftn(PSFproj, (I.shape[n] - 1,)))**2))
        z = z/np.max(z)
        beta[n] = np.append(z, z[0])
    
    if numDim == 1:
        alpha = 1 - beta[0]
    else: # n == 2:
        alpha = np.matmul((1-beta[0]).reshape((I.shape[0], 1)), (1 - beta[1]).reshape(1, I.shape[1]))
    # n > 2 - not implemented
    
    blurred = convolve2d(I, psf, 'same', 'symm')
    J = alpha * I + (1-alpha)*blurred

    return np.clip(J, np.min(I), np.max(I))

In [ ]:
psf = np.ones((20,20))
blurred = edgetaper(astro, psf)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(blurred, 'gray')
plt.show()